In [1]:
from solver_NerveNet import Solver
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from torchvision import transforms, datasets
from classifiers.NerveNet_U import NerveNET
import transform_utils_NerveNet as tu

from data_utils_NerveNet import SegmentationData

%load_ext autoreload
%autoreload 2

In [2]:
img_files = '../data/image_files.txt'
num_lines = sum(1 for line in open(img_files, 'r'))
nums = {'num_train' : int(0.6*num_lines)+2,
        'num_val'   : int(0.2*num_lines),
        'num_test' : int(0.2*num_lines)}

print(nums, num_lines)
print(np.sum(list(nums.values())) == num_lines)

{'num_train': 3383, 'num_val': 1127, 'num_test': 1127} 5635
False


In [3]:
input_dim = (1, 128, 128)
W, H = input_dim[1::]
W4, H4 = int(W / 2**3), int(H / 2**3)
W5, H5 = int(W / 2**4), int(H / 2**4)
resize_to_layer4 = (W4, H4)
resize_to_layer5 = (W5, H5)

transform1 = tu.Resize(input_dim[1::])
transform2 = tu.Resize(resize_to_layer4)
transform3 = tu.Resize(resize_to_layer5)
random_transforms = tu.Compose([
                        tu.RandomHorizontalFlip(),
                        tu.RandomVerticalFlip(),
                        tu.ToTensor()])

data_transform = [transform1, transform2, transform3, random_transforms]


train_data = SegmentationData(img_files, transform = data_transform, mode = 'train', **nums)
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=4,
                                           shuffle=True,
                                           num_workers=4)

val_data = SegmentationData(img_files,  transform = data_transform, mode = 'val', **nums)
val_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=4,
                                           shuffle=True,
                                           num_workers=4)

In [4]:
model = NerveNET(input_dim, num_classes = 2, weight_scale = 0.01, dropout = 0.15)
#model = torch.load("models/NerveNet_v1.model")

In [5]:
#for param in list(model.parameters()):
#    param.requires_grad = False
    
#for param in list(model.parameters())[-2::]:
#    param.requires_grad = True

solver = Solver(optim_args={"lr": 0.001, #0.0025, #1.e-3, #1.e-2
                            "betas": (0.9, 0.999),
                            "eps": 1e-8,
                            "weight_decay": 0.},
                loss_func = torch.nn.BCELoss(), loss_weights = [1.0, 0.1, 0.05, 0.01])

#print(model)
outputs = solver.train(model, train_loader, val_loader, log_nth=10, num_epochs=5)

START TRAIN.


/home/yy/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([4, 1, 128, 128])) that is different to the input size (torch.Size([4, 128, 128])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/yy/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([4, 1, 8, 8])) that is different to the input size (torch.Size([4, 8, 8])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/yy/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([4, 1, 16, 16])) that is different to the input size (torch.Size([4, 16, 16])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size())

[Iteration 10/4230] TRAIN loss: 0.804
[Iteration 20/4230] TRAIN loss: 0.791
[Iteration 30/4230] TRAIN loss: 0.767
[Iteration 40/4230] TRAIN loss: 0.723
[Iteration 50/4230] TRAIN loss: 0.658
[Iteration 60/4230] TRAIN loss: 0.578
[Iteration 70/4230] TRAIN loss: 0.491
[Iteration 80/4230] TRAIN loss: 0.409
[Iteration 90/4230] TRAIN loss: 0.337
[Iteration 100/4230] TRAIN loss: 0.287
[Iteration 110/4230] TRAIN loss: 0.250
[Iteration 120/4230] TRAIN loss: 0.222
[Iteration 130/4230] TRAIN loss: 0.197
[Iteration 140/4230] TRAIN loss: 0.170
[Iteration 150/4230] TRAIN loss: 0.181
[Iteration 160/4230] TRAIN loss: 0.162
[Iteration 170/4230] TRAIN loss: 0.161
[Iteration 180/4230] TRAIN loss: 0.162
[Iteration 190/4230] TRAIN loss: 0.149
[Iteration 200/4230] TRAIN loss: 0.145
[Iteration 210/4230] TRAIN loss: 0.132
[Iteration 220/4230] TRAIN loss: 0.157
[Iteration 230/4230] TRAIN loss: 0.150
[Iteration 240/4230] TRAIN loss: 0.142
[Iteration 250/4230] TRAIN loss: 0.132
[Iteration 260/4230] TRAIN loss: 0

/home/yy/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([3, 1, 128, 128])) that is different to the input size (torch.Size([3, 128, 128])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/yy/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([3, 1, 8, 8])) that is different to the input size (torch.Size([3, 8, 8])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/yy/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([3, 1, 16, 16])) that is different to the input size (torch.Size([3, 16, 16])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size())

[Epoch 1/5] TRAIN acc/loss: 0.288/0.119
[Epoch 1/5] VAL   acc/loss: 0.086/0.139
[Iteration 856/4230] TRAIN loss: 0.111
[Iteration 866/4230] TRAIN loss: 0.117
[Iteration 876/4230] TRAIN loss: 0.110
[Iteration 886/4230] TRAIN loss: 0.122
[Iteration 896/4230] TRAIN loss: 0.132
[Iteration 906/4230] TRAIN loss: 0.134
[Iteration 916/4230] TRAIN loss: 0.117
[Iteration 926/4230] TRAIN loss: 0.103
[Iteration 936/4230] TRAIN loss: 0.123
[Iteration 946/4230] TRAIN loss: 0.109
[Iteration 956/4230] TRAIN loss: 0.133
[Iteration 966/4230] TRAIN loss: 0.128
[Iteration 976/4230] TRAIN loss: 0.118
[Iteration 986/4230] TRAIN loss: 0.099
[Iteration 996/4230] TRAIN loss: 0.110
[Iteration 1006/4230] TRAIN loss: 0.116
[Iteration 1016/4230] TRAIN loss: 0.120
[Iteration 1026/4230] TRAIN loss: 0.110
[Iteration 1036/4230] TRAIN loss: 0.113
[Iteration 1046/4230] TRAIN loss: 0.127
[Iteration 1056/4230] TRAIN loss: 0.113
[Iteration 1066/4230] TRAIN loss: 0.097
[Iteration 1076/4230] TRAIN loss: 0.100
[Iteration 1086

[Iteration 2868/4230] TRAIN loss: 0.104
[Iteration 2878/4230] TRAIN loss: 0.122
[Iteration 2888/4230] TRAIN loss: 0.101
[Iteration 2898/4230] TRAIN loss: 0.101
[Iteration 2908/4230] TRAIN loss: 0.094
[Iteration 2918/4230] TRAIN loss: 0.091
[Iteration 2928/4230] TRAIN loss: 0.107
[Iteration 2938/4230] TRAIN loss: 0.113
[Iteration 2948/4230] TRAIN loss: 0.084
[Iteration 2958/4230] TRAIN loss: 0.106
[Iteration 2968/4230] TRAIN loss: 0.109
[Iteration 2978/4230] TRAIN loss: 0.097
[Iteration 2988/4230] TRAIN loss: 0.124
[Iteration 2998/4230] TRAIN loss: 0.101
[Iteration 3008/4230] TRAIN loss: 0.108
[Iteration 3018/4230] TRAIN loss: 0.108
[Iteration 3028/4230] TRAIN loss: 0.100
[Iteration 3038/4230] TRAIN loss: 0.123
[Iteration 3048/4230] TRAIN loss: 0.103
[Iteration 3058/4230] TRAIN loss: 0.116
[Iteration 3068/4230] TRAIN loss: 0.093
[Iteration 3078/4230] TRAIN loss: 0.124
[Iteration 3088/4230] TRAIN loss: 0.096
[Iteration 3098/4230] TRAIN loss: 0.099
[Iteration 3108/4230] TRAIN loss: 0.092


In [12]:
model.save("models/NerveNet_U_v1.model")

Saving model... models/NerveNet_U_v1.model


FileNotFoundError: [Errno 2] No such file or directory: 'models/NerveNet_U_v1.model'

In [7]:
test_data_transform = transforms.Resize(input_dim[1::])
img_files = '../data/image_files.txt'


test_data = SegmentationData(img_files,  transform = data_transform, mode = 'test', **nums)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=12,
                                          shuffle=False,
                                          num_workers=1)

FileNotFoundError: [Errno 2] No such file or directory: '../data/clean_no_zero_full_dataset_1300.txt'

In [ ]:
img, targets = test_data[9]

target = targets['in']
inputs = img.unsqueeze(0)
inputs = Variable(inputs)

outputs = model.forward(inputs)
pred = outputs[0].squeeze().data.cpu().numpy()

img=np.squeeze(img)
target = target.squeeze().numpy()

fig, ax = plt.subplots(1, 3)
ax[0].imshow(img)
ax[0].axis('off')
ax[1].imshow(target)
ax[1].axis('off')
ax[2].imshow(np.where(pred < 0.1, 0, 1))
ax[2].axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3)
ax[0].imshow(img)
ax[0].axis('off')
ax[1].imshow(target)
ax[1].axis('off')
ax[2].imshow(np.where(pred < 0.5, 0, 1))
ax[2].axis('off')
plt.show()

In [ ]:
np.where(pred < 0.5, 0, 1)

In [ ]:
num_example_imgs = 4
plt.figure(figsize=(15, 5 * num_example_imgs))
for i, (img, targets) in enumerate(test_data[1:4]):
    
    target = targets['in']
    inputs = img.unsqueeze(0)
    inputs = Variable(inputs)

    outputs = model.forward(inputs)
    pred = outputs[0].squeeze().data.cpu().numpy()

    img=np.squeeze(img)
    target = target.squeeze().numpy()
        
    # img
    plt.subplot(num_example_imgs, 3, i * 3 + 1)
    plt.axis('off')
    plt.imshow(img)
    if i == 0:
        plt.title("Input image")
    
    # target
    plt.subplot(num_example_imgs, 3, i * 3 + 2)
    plt.axis('off')
    plt.imshow(target)
    if i == 0:
        plt.title("Target image")

    # pred
    plt.subplot(num_example_imgs, 3, i * 3 + 3)
    plt.axis('off')
    plt.imshow(pred)
    if i == 0:
        plt.title("Prediction image")
    
plt.show()

In [ ]:
def dice_coefficient(ground_truth, predicted):
    gt = ground_truth
    p = predicted
    if np.sum(p) + np.sum(gt) == 0:
        return 1
    else:
        dice = np.sum(p[gt==1])*2.0 / (np.sum(p) + np.sum(gt))
        return dice

test_scores = []
model.eval()
for inputs, targets in test_loader:
    inputs, targets = Variable(inputs), Variable(targets['in'])
    if model.is_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    
    outputs = model.forward(inputs)
    preds = outputs[0]
    test_scores.append(dice_coefficient(np.squeeze(targets.data.numpy()), np.squeeze(preds.data.numpy())))#np.mean((preds == targets)[targets_mask].data.cpu().numpy()))
    
model.train()
np.mean(test_scores)